In [1]:
import os

from report_util import *
from trial import extract

WARMUP_ITERATIONS = 5
reports_dir = '/home/katie/Downloads/galette/slurm-1190681/'
pd.set_option('display.max_rows', 100)
data = extract(reports_dir, os.path.join(reports_dir, 'performance.csv'))
data.head()

Searching for trials in /home/katie/Downloads/galette/slurm-1190681/.
	Found 83 trials.
Checking trials.
	Missing required files for 73.
	Missing required files for 76.
	Missing required files for 59.
	Missing required files for 47.
	Missing required files for 79.
	Missing required files for 83.
	Missing required files for 61.
	Missing required files for 66.
	Missing required files for 45.
	Missing required files for 56.
	Missing required files for 51.
	Missing required files for 49.
	Missing required files for 55.
	Missing required files for 70.
	Missing required files for 57.
	Missing required files for 81.
	Missing required files for 46.
	Missing required files for 52.
	Missing required files for 80.
	Missing required files for 64.
	Missing required files for 60.
	Missing required files for 72.
	Missing required files for 53.
	60 trials were valid.
Creating combined experiment CSV.
	Wrote combined experiment CSV to /home/katie/Downloads/galette/slurm-1190681/performance.csv.


,iteration,rss,elapsed_time,benchmark,tool
0,5,1087200,478,spring,phosphor
1,6,1089604,462,spring,phosphor
2,7,1092956,526,spring,phosphor
3,8,1095584,421,spring,phosphor
4,9,1096460,430,spring,phosphor


In [2]:
import numpy as np


def create_table_frame(data, benchmark, y):
    selected = select(data, benchmark=benchmark)
    tools = sorted(data['tool'].unique())
    rows = []
    baseline_values = select(selected, tool='galette')[y]
    sig_level = compute_sig_level(3)
    for tool in tools:
        values = select(selected, tool=tool)[y]
        if len(values) > 0:
            p = mann_whitney(baseline_values, values)
            median = np.median(values)
            sig = 'color: red;' if p < sig_level else ''
            effect_size = a12(baseline_values, values)
            rows.append([tool, benchmark, median, p, sig, effect_size, len(values)])
        else:
            rows.append([tool, benchmark, np.NaN, np.NaN, np.NaN, np.NaN, 0])
    return pd.DataFrame(rows, columns=['tool', 'benchmark', 'med', 'p', 'sig', 'a12', 'n'])


def create_table(data, y):
    benchmarks = sorted(data['benchmark'].unique())
    return pd.concat(create_table_frame(data, benchmark, y) for benchmark in benchmarks)


def format_column_names(names):
    return (
        names[0].title().replace('-', '').replace('None', 'Baseline'),
        names[1]
    )


def create_performance_table(data, y, title):
    raw_table = create_table(data, y)
    raw_table['med'] = raw_table['med'].astype(pd.Int64Dtype())
    table = raw_table.pivot(index=['benchmark'], values=['med', 'p', 'a12'], columns=['tool']) \
        .reorder_levels(axis=1, order=['tool', None]) \
        .sort_index(axis=1) \
        .sort_index(axis=0) \
        .reindex(['galette', 'none', 'mirror-taint', 'phosphor'], axis=1, level=0) \
        .reindex(['med', 'p', 'a12'], axis=1, level=1)
    # Remove comparison of Galette against itself
    table = table.drop(columns=[('galette', 'p'), ('galette', 'a12')])
    table.index.names = [None for _ in table.index.names]
    table.index = table.index.map(str.title)
    table.columns = table.columns.map(format_column_names)
    table.columns.names = [None for _ in table.columns.names]
    return table.style.format(precision=3, na_rep='---') \
        .set_caption(title)

In [3]:
create_performance_table(data, 'rss', 'Memory Usage (kB).')

ValueError: `x` and `y` must be of nonzero size.

In [ ]:
create_performance_table(data, 'elapsed_time', 'Execution Time (ms).')